# Tải dataset và cài đặt các thành phần phụ thuộc 

In [0]:

!wget http://188.138.127.15:81/Datasets/Market-1501-v15.09.15.zip
!unzip Market-1501-v15.09.15.zip
!mv Market-1501-v15.09.15/ Market/
!pip install pretrainedmodels



In [0]:
!rm -rf /content/Person_reID_baseline_pytorch
!cd /content/ && git clone https://github.com/phamtrongthang123/Person_reID_baseline_pytorch

In [0]:
import os 
os.chdir('/content/Person_reID_baseline_pytorch')

In [0]:
!mv /content/net_last.pth /content/Person_reID_baseline_pytorch/model/PCB/net_last.pth

# Chạy prepare.py để tách các hình ảnh từ dataset vào đúng id để phục vụ cho dataloader

In [0]:
!python prepare.py

In [0]:
# train lâu, dùng model pretrained trước
#!python train.py --gpu_ids 0  --train_all --batchsize 32 --erasing_p 0.5 --lr 0.02

In [0]:
# chạy quá ram
#!python test.py --gpu_ids 0 --name PCB --batchsize 64

In [0]:
# Cài file này để download qua gg drive
!wget -O /usr/sbin/gdrivedl 'https://f.mjh.nz/gdrivedl'
!chmod +x /usr/sbin/gdrivedl

--2019-06-01 13:44:20--  https://f.mjh.nz/gdrivedl
Resolving f.mjh.nz (f.mjh.nz)... 104.28.30.233, 104.28.31.233, 2606:4700:30::681c:1fe9, ...
Connecting to f.mjh.nz (f.mjh.nz)|104.28.30.233|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1362 (1.3K) [application/octet-stream]
Saving to: ‘/usr/sbin/gdrivedl’

/usr/sbin/gdrivedl  100%[===================>]   1.33K  --.-KB/s    in 0s      

2019-06-01 13:44:20 (34.4 MB/s) - ‘/usr/sbin/gdrivedl’ saved [1362/1362]



In [0]:
!gdrivedl https://drive.google.com/uc?id=1XVEYb0TN2SbBYOqf8SzazfYZlpH9CxyE&export=download

File ID: 1XVEYb0TN2SbBYOqf8SzazfYZlpH9CxyE
Downloading: https://docs.google.com/uc?export=download&id=1XVEYb0TN2SbBYOqf8SzazfYZlpH9CxyE > .246.file
Downloading: https://docs.google.com/uc?export=download&id=1XVEYb0TN2SbBYOqf8SzazfYZlpH9CxyE&confirm=aMoX > .246.file
Moving: .246.file > model.zip
Saved: model.zip
DONE!


In [0]:
!unzip model.zip

Archive:  model.zip
  inflating: model/ft_ResNet50/train.jpg  
replace model/ft_ResNet50/model.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: model/ft_ResNet50/model.py  
  inflating: model/ft_ResNet50/_DS_Store  
  inflating: model/ft_ResNet50/opts.yaml  
  inflating: model/ft_ResNet50/train.py  
  inflating: model/ft_ResNet50/net_last.pth  
 extracting: model/_gitkeep          
  inflating: model/_DS_Store         
 extracting: model/.gitkeep          
  inflating: model/fp16/train.jpg    
  inflating: model/fp16/model.py     
  inflating: model/fp16/opts.yaml    
  inflating: model/fp16/train.py     
  inflating: model/fp16/net_last.pth  
  inflating: model/ft_net_dense/train.jpg  
  inflating: model/ft_net_dense/model.py  
  inflating: model/ft_net_dense/_DS_Store  
  inflating: model/ft_net_dense/opts.yaml  
  inflating: model/ft_net_dense/train.py  
  inflating: model/ft_net_dense/net_last.pth  
  inflating: model/PCB/train.jpg     
  inflating: model/PCB/model.py      

In [0]:
!rm -rf /content/Person_reID_baseline_pytorch/model && mv /content/model /content/Person_reID_baseline_pytorch/model

# Chỉnh lại code test.py để chạy được

Chỉnh tại phần dataloader, file gốc num worker là 32, giảm còn 8, batch sie gốc là 256, giảm còn 128 thì chạy được

In [0]:
# -*- coding: utf-8 -*-

from __future__ import print_function, division

import argparse
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import scipy.io
import yaml
from model import ft_net, ft_net_dense, ft_net_NAS, PCB, PCB_test



str_ids = '0'
#which_epoch = opt.which_epoch
name = 'PCB'
test_dir = '/content/Market/pytorch'

gpu_ids = []
for str_id in str_ids:
    id = int(str_id)
    if id >=0:
        gpu_ids.append(id)

# set gpu ids
if len(gpu_ids)>0:
    torch.cuda.set_device(gpu_ids[0])
    cudnn.benchmark = True

######################################################################
# Load Data
# ---------
#
# We will use torchvision and torch.utils.data packages for loading the
# data.
#
data_transforms = transforms.Compose([
        transforms.Resize((256,128), interpolation=3),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
############### Ten Crop        
        #transforms.TenCrop(224),
        #transforms.Lambda(lambda crops: torch.stack(
         #   [transforms.ToTensor()(crop) 
          #      for crop in crops]
           # )),
        #transforms.Lambda(lambda crops: torch.stack(
         #   [transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(crop)
          #       for crop in crops]
          # ))
])


data_transforms = transforms.Compose([
    transforms.Resize((384,192), interpolation=3),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
])


data_dir = test_dir


image_datasets = {x: datasets.ImageFolder( os.path.join(data_dir,x) ,data_transforms) for x in ['gallery','query']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=128,
                                       shuffle=False, num_workers=8) for x in ['gallery','query']}
class_names = image_datasets['query'].classes
use_gpu = torch.cuda.is_available()

######################################################################
# Load model
#---------------------------
def load_network(network):
    save_path = '/content/Person_reID_baseline_pytorch/model/PCB/net_last.pth'
    network.load_state_dict(torch.load(save_path))
    return network


######################################################################
# Extract feature
# ----------------------
#
# Extract feature from  a trained model.
#
def fliplr(img):
    '''flip horizontal'''
    inv_idx = torch.arange(img.size(3)-1,-1,-1).long()  # N x C x H x W
    img_flip = img.index_select(3,inv_idx)
    return img_flip

def extract_feature(model,dataloaders):
    features = torch.FloatTensor()
    count = 0
    for data in dataloaders:
        img, label = data
        n, c, h, w = img.size()
        count += n
        print(count)
        ff = torch.FloatTensor(n,512).zero_()

       
        ff = torch.FloatTensor(n,2048,6).zero_() # we have six parts
        for i in range(2):
            if(i==1):
                img = fliplr(img)
            input_img = Variable(img.cuda())
            #if opt.fp16:
            #    input_img = input_img.half()
            outputs = model(input_img) 
            f = outputs.data.cpu().float()
            ff = ff+f

        # feature size (n,2048,6)
        # 1. To treat every part equally, I calculate the norm for every 2048-dim part feature.
        # 2. To keep the cosine score==1, sqrt(6) is added to norm the whole feature (2048*6).
        fnorm = torch.norm(ff, p=2, dim=1, keepdim=True) * np.sqrt(6) 
        ff = ff.div(fnorm.expand_as(ff))
        ff = ff.view(ff.size(0), -1)


        features = torch.cat((features,ff), 0)
    return features

def get_id(img_path):
    camera_id = []
    labels = []
    for path, v in img_path:
        #filename = path.split('/')[-1]
        filename = os.path.basename(path)
        label = filename[0:4]
        camera = filename.split('c')[1]
        if label[0:2]=='-1':
            labels.append(-1)
        else:
            labels.append(int(label))
        camera_id.append(int(camera[0]))
    return camera_id, labels

gallery_path = image_datasets['gallery'].imgs
query_path = image_datasets['query'].imgs

gallery_cam,gallery_label = get_id(gallery_path)
query_cam,query_label = get_id(query_path)



######################################################################
# Load Collected data Trained model
print('-------test-----------')

model_structure = PCB(751)

#if opt.fp16:
#    model_structure = network_to_half(model_structure)

model = load_network(model_structure)


model = PCB_test(model)


# Change to test mode
model = model.eval()
if use_gpu:
    model = model.cuda()

# Extract feature
with torch.no_grad():
    gallery_feature = extract_feature(model,dataloaders['gallery'])
    query_feature = extract_feature(model,dataloaders['query'])

    
# Save to Matlab for check
result = {'gallery_f':gallery_feature.numpy(),'gallery_label':gallery_label,'gallery_cam':gallery_cam,'query_f':query_feature.numpy(),'query_label':query_label,'query_cam':query_cam}
scipy.io.savemat('pytorch_result.mat',result)


-------test-----------


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [00:03<00:00, 25693595.19it/s]


128
256
384
512
640
768
896
1024
1152
1280
1408
1536
1664
1792
1920
2048
2176
2304
2432
2560
2688
2816
2944
3072
3200
3328
3456
3584
3712
3840
3968
4096
4224
4352
4480
4608
4736
4864
4992
5120
5248
5376
5504
5632
5760
5888
6016
6144
6272
6400
6528
6656
6784
6912
7040
7168
7296
7424
7552
7680
7808
7936
8064
8192
8320
8448
8576
8704
8832
8960
9088
9216
9344
9472
9600
9728
9856
9984
10112
10240
10368
10496
10624
10752
10880
11008
11136
11264
11392
11520
11648
11776
11904
12032
12160
12288
12416
12544
12672
12800
12928
13056
13184
13312
13440
13568
13696
13824
13952
14080
14208
14336
14464
14592
14720
14848
14976
15104
15232
15360
15488
15616
15744
15872
16000
16128
16256
16384
16512
16640
16768
16896
17024
17152
17280
17408
17536
17664
17792
17920
18048
18176
18304
18432
18560
18688
18816
18944
19072
19200
19328
19456
19584
19712
19732
128
256
384
512
640
768
896
1024
1152
1280
1408
1536
1664
1792
1920
2048
2176
2304
2432
2560
2688
2816
2944
3072
3200
3328
3368


In [0]:
# chay xong thì eval model
!cd Person_reID_baseline_pytorch/ && python evaluate_gpu.py

torch.Size([3368, 12288])
Rank@1:0.926366 Rank@5:0.973278 Rank@10:0.981888 mAP:0.774712


# Dưới đây là phần sửa code demo để phục vụ việc xuất các file hình ảnh và lưu vào json để tiện xử lí sau này

Ở dưới em có mount đến drive cá nhân nên phần xuất ở dưới sẽ cần chỉnh lại path cho phù hợp.

Sau khi xuất ra thì em zip lại up thành file khác, link đến file nằm trong file notebook em dùng để phân tích.

In [0]:
import argparse
import scipy.io
import torch
import numpy as np
import os
from torchvision import datasets
import matplotlib
import matplotlib.pyplot as plt

matplotlib.use('agg')
import matplotlib.pyplot as plt
#######################################################################
# Evaluate


data_dir = '../Market/pytorch'
image_datasets = {x: datasets.ImageFolder( os.path.join(data_dir,x) ) for x in ['gallery','query']}

#####################################################################
#Show result
def imshow(path, title=None):
    """Imshow for Tensor."""
    im = plt.imread(path)
    plt.imshow(im)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

######################################################################
result = scipy.io.loadmat('pytorch_result.mat')
query_feature = torch.FloatTensor(result['query_f'])
query_cam = result['query_cam'][0]
query_label = result['query_label'][0]
gallery_feature = torch.FloatTensor(result['gallery_f'])
gallery_cam = result['gallery_cam'][0]
gallery_label = result['gallery_label'][0]

multi = os.path.isfile('multi_query.mat')

if multi:
    m_result = scipy.io.loadmat('multi_query.mat')
    mquery_feature = torch.FloatTensor(m_result['mquery_f'])
    mquery_cam = m_result['mquery_cam'][0]
    mquery_label = m_result['mquery_label'][0]
    mquery_feature = mquery_feature.cuda()

query_feature = query_feature.cuda()
gallery_feature = gallery_feature.cuda()

#######################################################################
# sort the images
def sort_img(qf, ql, qc, gf, gl, gc):
    query = qf.view(-1,1)
    # print(query.shape)
    score = torch.mm(gf,query)
    score = score.squeeze(1).cpu()
    score = score.numpy()
    # predict index
    index = np.argsort(score)  #from small to large
    index = index[::-1]
    # index = index[0:2000]
    # good index
    query_index = np.argwhere(gl==ql)
    #same camera
    camera_index = np.argwhere(gc==qc)

    #good_index = np.setdiff1d(query_index, camera_index, assume_unique=True)
    junk_index1 = np.argwhere(gl==-1)
    junk_index2 = np.intersect1d(query_index, camera_index)
    junk_index = np.append(junk_index2, junk_index1) 

    mask = np.in1d(index, junk_index, invert=True)
    index = index[mask]
    return index

### QUERY INDEX HERE
i = 777
index = sort_img(query_feature[i],query_label[i],query_cam[i],gallery_feature,gallery_label,gallery_cam)

########################################################################
# Visualize the rank result

query_path, _ = image_datasets['query'].imgs[i]
query_label = query_label[i]
print(query_path)
print('Top 10 images are as follow:')
try: # Visualize Ranking Result 
    # Graphical User Interface is needed
    fig = plt.figure(figsize=(16,4))
    ax = plt.subplot(1,11,1)
    ax.axis('off')
    imshow(query_path,'query')
    for i in range(10):
        ax = plt.subplot(1,11,i+2)
        ax.axis('off')
        img_path, _ = image_datasets['gallery'].imgs[index[i]]
        label = gallery_label[index[i]]
        imshow(img_path)
        if label == query_label:
            ax.set_title('%d'%(i+1), color='green')
        else:
            ax.set_title('%d'%(i+1), color='red')
        print(img_path)
except RuntimeError:
    for i in range(10):
        img_path = image_datasets.imgs[index[i]]
        print(img_path[0])
    print('If you want to see the visualization of the ranking result, graphical user interface is needed.')

fig.savefig("show.png")


../Market/pytorch/query/0342/0342_c5s1_079123_00.jpg
Top 10 images are as follow:
../Market/pytorch/gallery/0342/0342_c2s1_078546_01.jpg
../Market/pytorch/gallery/0342/0342_c3s1_078792_03.jpg
../Market/pytorch/gallery/0342/0342_c3s1_078767_01.jpg
../Market/pytorch/gallery/0342/0342_c2s1_078646_01.jpg
../Market/pytorch/gallery/0252/0252_c5s1_058423_03.jpg
../Market/pytorch/gallery/0342/0342_c1s2_022816_02.jpg
../Market/pytorch/gallery/0252/0252_c5s1_058448_06.jpg
../Market/pytorch/gallery/0252/0252_c1s1_052801_03.jpg
../Market/pytorch/gallery/0342/0342_c2s1_095146_01.jpg
../Market/pytorch/gallery/0252/0252_c3s1_052226_01.jpg


In [0]:
%cd /content/Person_reID_baseline_pytorch/
import argparse
import scipy.io
import torch
import numpy as np
import os
from torchvision import datasets
import matplotlib
import matplotlib.pyplot as plt

matplotlib.use('agg')
import matplotlib.pyplot as plt
#######################################################################
# Evaluate


data_dir = '../Market/pytorch'
image_datasets = {x: datasets.ImageFolder( os.path.join(data_dir,x) ) for x in ['gallery','query']}

#####################################################################
#Show result
def imshow(path, title=None):
    """Imshow for Tensor."""
    im = plt.imread(path)
    plt.imshow(im)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

######################################################################
result = scipy.io.loadmat('pytorch_result.mat')
query_feature = torch.FloatTensor(result['query_f'])
query_cam = result['query_cam'][0]
query_label = result['query_label'][0]
gallery_feature = torch.FloatTensor(result['gallery_f'])
gallery_cam = result['gallery_cam'][0]
gallery_label = result['gallery_label'][0]

multi = os.path.isfile('multi_query.mat')

if multi:
    m_result = scipy.io.loadmat('multi_query.mat')
    mquery_feature = torch.FloatTensor(m_result['mquery_f'])
    mquery_cam = m_result['mquery_cam'][0]
    mquery_label = m_result['mquery_label'][0]
    mquery_feature = mquery_feature.cuda()

query_feature = query_feature.cuda()
gallery_feature = gallery_feature.cuda()

#######################################################################
# sort the images
def sort_img(qf, ql, qc, gf, gl, gc):
    query = qf.view(-1,1)
    # print(query.shape)
    score = torch.mm(gf,query)
    score = score.squeeze(1).cpu()
    score = score.numpy()
    # predict index
    index = np.argsort(score)  #from small to large
    index = index[::-1]
    # index = index[0:2000]
    # good index
    query_index = np.argwhere(gl==ql)
    #same camera
    camera_index = np.argwhere(gc==qc)

    #good_index = np.setdiff1d(query_index, camera_index, assume_unique=True)
    junk_index1 = np.argwhere(gl==-1)
    junk_index2 = np.intersect1d(query_index, camera_index)
    junk_index = np.append(junk_index2, junk_index1) 

    mask = np.in1d(index, junk_index, invert=True)
    index = index[mask]
    return index

for i in range(3367+1 - 2300):
    i = i + 2300
    ### QUERY INDEX HERE
    #i = 777
    print(i)
    index = sort_img(query_feature[i],query_label[i],query_cam[i],gallery_feature,gallery_label,gallery_cam)

    ########################################################################
    # Visualize the rank result
    j = i
    query_path, _ = image_datasets['query'].imgs[i]
    #query_label = query_label[i]
    print(query_path)
    print('Top 10 images are as follow:')
    try: # Visualize Ranking Result 
        # Graphical User Interface is needed
        fig = plt.figure(figsize=(16,4))
        ax = plt.subplot(1,11,1)
        ax.axis('off')
        imshow(query_path,'query')
        for i in range(10):
            ax = plt.subplot(1,11,i+2)
            ax.axis('off')
            img_path, _ = image_datasets['gallery'].imgs[index[i]]
            label = gallery_label[index[i]]
            imshow(img_path)
            if label == query_label[j]:
                ax.set_title('%d'%(i+1), color='green')
            else:
                ax.set_title('%d'%(i+1), color='red')
            print(img_path)
    except RuntimeError:
        for i in range(10):
            img_path = image_datasets.imgs[index[i]]
            print(img_path[0])
        print('If you want to see the visualization of the ranking result, graphical user interface is needed.')
    if not os.path.exists('./resultimgs'):
        os.makedirs('./resultimgs')
    fig.savefig("./resultimgs/show%d.png"%j)
    i = j
    

../Market/pytorch/gallery/1352/1352_c2s3_034882_01.jpg
../Market/pytorch/gallery/1352/1352_c3s3_040653_10.jpg
../Market/pytorch/gallery/1268/1268_c3s3_025178_02.jpg
../Market/pytorch/gallery/1268/1268_c6s3_066442_01.jpg
../Market/pytorch/gallery/1268/1268_c2s3_036982_05.jpg
../Market/pytorch/gallery/0405/0405_c3s2_056162_02.jpg
3007
../Market/pytorch/query/1352/1352_c2s3_034732_00.jpg
Top 10 images are as follow:
../Market/pytorch/gallery/1352/1352_c3s3_040703_01.jpg
../Market/pytorch/gallery/1352/1352_c3s3_040653_10.jpg
../Market/pytorch/gallery/1352/1352_c4s5_066210_01.jpg
../Market/pytorch/gallery/1352/1352_c1s5_067716_02.jpg
../Market/pytorch/gallery/1352/1352_c1s5_067016_01.jpg
../Market/pytorch/gallery/1352/1352_c1s5_067041_02.jpg
../Market/pytorch/gallery/1352/1352_c5s3_040940_04.jpg
../Market/pytorch/gallery/0989/0989_c6s2_120543_01.jpg
../Market/pytorch/gallery/0989/0989_c6s2_120518_01.jpg
../Market/pytorch/gallery/0463/0463_c1s2_046021_01.jpg
3008
../Market/pytorch/query/1352

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
%cd /content/gdrive/My\ Drive/RandomScript/PReID

import argparse
import scipy.io
import torch
import numpy as np
import os
from torchvision import datasets
import matplotlib
import matplotlib.pyplot as plt

matplotlib.use('agg')
import matplotlib.pyplot as plt
#######################################################################
# Evaluate


data_dir = '../Market/pytorch'
image_datasets = {x: datasets.ImageFolder( os.path.join(data_dir,x) ) for x in ['gallery','query']}

#####################################################################
#Show result
def imshow(path, title=None):
    """Imshow for Tensor."""
    im = plt.imread(path)
    plt.imshow(im)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

######################################################################
result = scipy.io.loadmat('pytorch_result.mat')
query_feature = torch.FloatTensor(result['query_f'])
query_cam = result['query_cam'][0]
query_label = result['query_label'][0]
gallery_feature = torch.FloatTensor(result['gallery_f'])
gallery_cam = result['gallery_cam'][0]
gallery_label = result['gallery_label'][0]

multi = os.path.isfile('multi_query.mat')

if multi:
    m_result = scipy.io.loadmat('multi_query.mat')
    mquery_feature = torch.FloatTensor(m_result['mquery_f'])
    mquery_cam = m_result['mquery_cam'][0]
    mquery_label = m_result['mquery_label'][0]
    mquery_feature = mquery_feature.cuda()

query_feature = query_feature.cuda()
gallery_feature = gallery_feature.cuda()

#######################################################################
# sort the images
def sort_img(qf, ql, qc, gf, gl, gc):
    query = qf.view(-1,1)
    # print(query.shape)
    score = torch.mm(gf,query)
    score = score.squeeze(1).cpu()
    score = score.numpy()
    # predict index
    index = np.argsort(score)  #from small to large
    index = index[::-1]
    # index = index[0:2000]
    # good index
    query_index = np.argwhere(gl==ql)
    #same camera
    camera_index = np.argwhere(gc==qc)

    #good_index = np.setdiff1d(query_index, camera_index, assume_unique=True)
    junk_index1 = np.argwhere(gl==-1)
    junk_index2 = np.intersect1d(query_index, camera_index)
    junk_index = np.append(junk_index2, junk_index1) 

    mask = np.in1d(index, junk_index, invert=True)
    index = index[mask]
    return index

for i in range(3367+1 - 2300):
    i = i + 2300
    ### QUERY INDEX HERE
    #i = 777
    print(i)
    index = sort_img(query_feature[i],query_label[i],query_cam[i],gallery_feature,gallery_label,gallery_cam)

    ########################################################################
    # Visualize the rank result
    j = i
    query_path, _ = image_datasets['query'].imgs[i]
    #query_label = query_label[i]
    print(query_path)
    print('Top 10 images are as follow:')
    try: # Visualize Ranking Result 
        # Graphical User Interface is needed
        fig = plt.figure(figsize=(16,4))
        ax = plt.subplot(1,11,1)
        ax.axis('off')
        imshow(query_path,'query')
        for i in range(10):
            ax = plt.subplot(1,11,i+2)
            ax.axis('off')
            img_path, _ = image_datasets['gallery'].imgs[index[i]]
            label = gallery_label[index[i]]
            imshow(img_path)
            if label == query_label[j]:
                ax.set_title('%d'%(i+1), color='green')
            else:
                ax.set_title('%d'%(i+1), color='red')
            print(img_path)
    except RuntimeError:
        for i in range(10):
            img_path = image_datasets.imgs[index[i]]
            print(img_path[0])
        print('If you want to see the visualization of the ranking result, graphical user interface is needed.')
    if not os.path.exists('./resultimgs'):
        os.makedirs('./resultimgs')
    fig.savefig("./resultimgs/show%d.png"%j)
    i = j
    